<a href="https://colab.research.google.com/github/carolina-garay/AnalisisYCuracion_Grupo32/blob/Seba-Ormaechea/Pr%C3%A1ctico2_AyC_Parte1_SGO_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2022*

----

# Trabajo práctico N°2. Análisis y Curación de datos. Entregable - parte 1

El objetivo de este práctico es: Obtener una base de datos capaz de ser utilizada posteriormente para la construcción de un modelo que prediga el valor de venta de una propiedad en la ciudad de Melbourne, Australia. Para ello, se cargan dos conjuntos de datos. El [primero](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) es de la compentencia Kaggle sobre estimación de precios de ventas de propiedades en Melbourne, Australia. El [segundo](https://www.kaggle.com/datasets/tylerx/melbourne-airbnb-open-data) es un conjunto de datos  elaborado a partir de datos de la plataforma Airbnb, que se buscará unir al primer dataset con el objetivo de sumarle información.

In [21]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn
seaborn.set_context('talk')

#from decouple import config
from sqlalchemy import create_engine, text
import plotly
plotly.__version__

'5.5.0'

## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite



In [4]:
# Creamos la conexión a la base de datos
engine = create_engine('sqlite:///sysarmy.sqlite3', echo=True)

2. Ingestamos los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

In [ ]:
# Ingesta primer conjunto de datos y creamos una base de datos nueva para ser usada con lenguaje SQL
kaggle_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
kaggle_df.to_sql('kaggle_sql', con=engine, if_exists="replace")

In [19]:
# Ingesta de datos de dataset generado en clase con datos de Airbnb
from google.colab import files
uploaded = files.upload()


Saving airbnb_price_by_zipcode.csv to airbnb_price_by_zipcode.csv


In [25]:
airbnb_price_by_zipcode = pd.read_csv(io.StringIO(uploaded['airbnb_price_by_zipcode.csv'].decode('utf-8')))

In [ ]:
airbnb_price_by_zipcode.to_sql('airbnb_price_sql', con=engine, if_exists="replace")

3. Implementamos consultas en SQL que responden a la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

In [ ]:
# Cantidad de registros totales por ciudad en dataset Kaggle
query1 = "SELECT CouncilArea, COUNT(CouncilArea) FROM kaggle_sql GROUP BY CouncilArea"
          
# Cantidad de registros totales por barrio y ciudad en Kaggle
query2 = "SELECT CouncilArea, Suburb, COUNT(Suburb) FROM kaggle_sql GROUP BY Suburb ORDER BY CouncilArea"

# Listamos los queries de consulta
queries = [query1, query2]

# Conectamos a la base de datos y sistematizamos las consultas con un bucle For
with engine.connect() as con:
    for query in queries:
      rs = con.execute(query)
      print(query)
      for row in rs:
          print(row)
      print('\n\n')


4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.

In [ ]:
Both_Datasets= engine.connect().execute('SELECT * FROM kaggle_sql INNER JOIN airbnb_price_sql ON kaggle_sql.Postcode = airbnb_price_sql.zipcode')

## Ejercicio 2: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)